In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
from silkpy.surface import ParametricSurface
from sympy import symbols, sin, cos, pi, cot, Array
u, v = symbols('u, v', real=True)

In [7]:
R = symbols('R', positive=True)
canonical_cylindricl_surface = ParametricSurface([R*cos(u), R*sin(u), v], [u, v])

In [8]:
w = symbols('omega', real=True)
canonical_cone_surface = ParametricSurface([v*cos(u), v*sin(u), v*cot(w)], [u, v])

In [9]:
theta = symbols('theta', real=True)
Mobius_strip = ParametricSurface(
    Array([cos(theta), 
           sin(theta), 
           0]) + 
    v * Array([sin(theta/2) * cos(theta), 
               sin(theta/2) * sin(theta), 
               cos(theta/2)]), [u, v])